In [0]:
import numpy as np
np.random.seed(seed=484)
import warnings
warnings.filterwarnings('ignore')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Midterm Exam

Exam is open book, open note, and open Google. You are not allowed outside
help from another person, however. All work, including coding, must be yours alone. Remember to turn in both the written portion and this coding portion. To complete this coding portion, supply the python code in the empty cells below, and execute the notebook. To get full credit, the completed notebook should be able to run top to bottom, producing the results asked for in the prompts below.

This portion of the exam will take you through the steps of the supervised machine learning process.

## 1. Figure out your question

The question you want to answer using machine learning is: Would two people with identical observed characteristics but different race be predicted to have their loan applications treated differently?

## 2. Obtain a labeled dataset

Import the python library that is good for manipulating datasets:

In [0]:
import pandas as pd
#import numpy as np

Accompanying the exam materials are a spreadsheet of mortgage loan applicants, 'mortgagedenials.csv' and a text file, 'variabledefs.txt' that explains each variable in the spreadsheet. Read in the data in the spreadsheet 'mortgagedenials.csv', print out the first few rows of data with the variable names, and print out the number of observations and variables in the dataset:

In [0]:
#vars = pd.read_csv("/content/drive/My Drive/variabledefs.txt", delimiter="\t")
#print(vars.columns)
var = pd.read_csv("/content/drive/My Drive/variabledefs.txt", 
                   sep = "\t")
#var.columns = ["variable name", "description"]
mort = pd.read_csv("/content/drive/My Drive/mortgagedenials.csv")

### come back to this cell 


Define a label (outcome) vector, $y$, to be an indicator for whether the individual's mortgage application was denied, and define a feature (regressor) matrix, $X$, to be all remaining columns:

In [7]:
#finding my y variable
mort.head()
#selecting it as y 
y = mort.deny
#selecting other columns to be the columns used for the data set 
data = mort.drop(axis = 1, columns = "deny")
data.head()

,p_irat,black,hse_inc,loan_val,ccred,mcred,pubrec,denpmi,selfemp,single,hischl,probunmp,condo,ltv_med,ltv_high
0,0.217,0,0.213,0.638889,1,1,0,0,0,0,1,3.2,0,0,0
1,0.260,0,0.220,0.945946,6,2,0,0,0,0,1,3.9,0,1,0
2,0.460,0,0.270,0.840000,5,2,0,0,0,1,1,5.3,0,1,0
3,0.490,0,0.270,0.199482,1,1,0,0,0,0,1,3.2,0,0,0
4,0.260,0,0.260,0.366667,1,2,0,0,0,0,1,4.3,0,0,0


## 3. Divide into training and set sets

Import the python library that is good for randomly splitting datasets into training and test sets:

In [0]:
#importing library from sklearn 
from sklearn.model_selection import train_test_split, GridSearchCV

Now make a training and test feature matrix and a training and test label vector:

In [0]:
#splitting 
X_train, X_test, y_train, y_test = train_test_split(data, y,
                                                   test_size = 0.33,
                                                   random_state = 42)

## 4. Pick an appropriate method

Choose a method appropriate for classification and import its library:

In [0]:
''' I am going to utilize a random forest classifier, this will give me a 0
or 1 outcome based on if the person was denied or not''' 
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from sklearn.metrics import f1_score, classification_report

## 5. Choose regularization parameters via cross-validation on the training set

Search over a grid of values of the regularization parameters for the parameters that perform the best out of sample:

In [11]:
#building a param dictionary with different tree depths, and number of trees
#creating a dictionary with multiple options for the parameters, 
param_dict = {"n_estimators":[100,500,1000,2500],
              "max_depth":[1,3,5,10]}
            #   "min_samples_split":[6,8,10,12],
            #   "min_samples_leaf":[4,5,6,7]}

#creating a class object 
rf = RandomForestClassifier()  

#creating the grid search to search over my parameters and find the best ones 
gs = GridSearchCV(rf, param_dict, scoring = "f1_macro", 
                  n_jobs = -1, verbose = 2)
gs.fit(X_train, y_train)

#printing the best parameters 
print("Best Params: {}".format(gs.best_params_))

#ok so looks like best params are max depth of 10 and number of tree 100 

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   42.0s
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  1.0min finished


Best Params: {'max_depth': 10, 'n_estimators': 1000}


## 6. Fit model on whole training set using the cross-validated parameters

In [0]:
rf.fit(X_train, y_train)
forest_pred = rf.predict(X_test)

## 7. Evaluate model by applying it to test set

Compute and print out the "score" of the model applied to the test set:

In [13]:
print("Random Forest Classification Report:")
print(classification_report(y_test, forest_pred))
from sklearn.metrics import confusion_matrix
y_pred = gs.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)

#ok so I am vastly over fitting, my best parameters are max depth of 10 but
#only 100 trees, so I'll go again, closer around that neighborhood to see if
# i can improve, from the confusion matrix you can clearly see the missclasification 


Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.98      0.95       697
           1       0.68      0.29      0.41        89

    accuracy                           0.90       786
   macro avg       0.80      0.64      0.68       786
weighted avg       0.89      0.90      0.89       786

[[684  13]
 [ 61  28]]


In [14]:
"""I want to see what happens if ai narrow my dictionary around my best params
"""

param_dict = {"n_estimators":[200, 300, 400, 500, 600],
              "max_depth":[8, 10, 12, 14]}

gs2 = GridSearchCV(rf, param_dict, scoring = "f1_macro", 
                  n_jobs = -1, verbose = 2)
gs2.fit(X_train, y_train)

#printing the best parameters 
print("Best Params: {}".format(gs2.best_params_))


rf.fit(X_train, y_train)
y_pred2 = rf.predict(X_test)
print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred2))


from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred2)
print(cm)
#this was slightly better in terms of f1, my confusion matrix is also here 
# showing that I am still mis classifying a lot 

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   19.4s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   33.9s finished


Best Params: {'max_depth': 12, 'n_estimators': 300}
Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.98      0.94       697
           1       0.62      0.26      0.37        89

    accuracy                           0.90       786
   macro avg       0.77      0.62      0.65       786
weighted avg       0.88      0.90      0.88       786

[[683  14]
 [ 66  23]]


## 8. Repeat 4-7 for another method

Import the method's library, do cross validation to find tuning parameters, fit the model on the training data using the cross-validated tuning parameters, and compute (and report) the model's score on the test set:

In [15]:
#before I begin, I want to see what the number of yes vs no is 

y.value_counts()
#ok so i have 2095 yes and 285 = 2380 total 
#and only 11% of those are no, I might synthetically create some no data to
#make it more even 

0    2095
1     285
Name: deny, dtype: int64

In [16]:
#now we will use support vector machines to classify the data in our second attempt
from sklearn.svm import SVC
svc = SVC(kernel = "linear")

#creating dictionary
sv_params = {"C": [0.03, 0.06, .09, .12, 0.15], 
             #"kernel": ['linear'], 
             'gamma': [0.3, .6, .9, 1.2, 1.5]}


gridsearch = GridSearchCV(svc, 
                          sv_params, 
                          scoring='f1')

gridsearch.fit(X_train, y_train)
print("Best Params: {}".format(gridsearch.best_params_))

y_pred = gridsearch.predict(X_test)

#getting accuracy metrics 
from sklearn.metrics import f1_score, classification_report
report = classification_report(y_test, y_pred)
print(report)

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
print(cm)

Best Params: {'C': 0.09, 'gamma': 0.3}
              precision    recall  f1-score   support

           0       0.90      1.00      0.95       697
           1       1.00      0.15      0.25        89

    accuracy                           0.90       786
   macro avg       0.95      0.57      0.60       786
weighted avg       0.91      0.90      0.87       786

[[697   0]
 [ 76  13]]


In [17]:
#repeating for polynomial kernel 

svc = SVC(kernel = "poly", degree = 2)
sv_params = {"C": [0.03, 0.06, .09, .12, 0.15], 
             #"kernel": ['precomputed'],
             'gamma': [0.3, .6, .9, 1.2, 1.5]}


gridsearch = GridSearchCV(svc, 
                          sv_params, 
                          scoring='f1',
                          cv = 3
                          )

gridsearch.fit(X_train, y_train)

y_pred = gridsearch.predict(X_test)
#print("Best Params: {}".format(gridsearch.best_params_))
#getting accuracy metrics 
from sklearn.metrics import f1_score, classification_report
report = classification_report(y_test, y_pred)
print(report)

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
print(cm)

'''still mis classifying a lot.  Tips to help with over fitting:
increase the amount of training data, maybe use k fold cross validation so that
we maximize the amount of information.  also we can decraese the number of parameters
and maybe synthetically create more rejection observations. We have almost 90 percent 
of the observations are yes got approved, so the computer is having a hard time
because if it simply says, yes they got approved, its right 90% of the time. 
'''

              precision    recall  f1-score   support

           0       0.91      0.98      0.94       697
           1       0.61      0.28      0.38        89

    accuracy                           0.90       786
   macro avg       0.76      0.63      0.66       786
weighted avg       0.88      0.90      0.88       786

[[681  16]
 [ 64  25]]


'still mis classifying a lot.  Tips to help with over fitting:\nincrease the amount of training data, maybe use k fold cross validation so that\nwe maximize the amount of information.  also we can decraese the number of parameters\nand maybe synthetically create more rejection observations. We have almost 90 percent \nof the observations are yes got approved, so the computer is having a hard time\nbecause if it simply says, yes they got approved, its right 90% of the time. \n'

## 9. Apply the chosen method to new observations for which we have no labels

Fit the method you chose (based on performance on the test set) to your entire dataset:

In [18]:
'''
ok so my weighted average of precision and recall is my f1 score and that was 
highest in the random forest.  When I narrowed my scope around my best params 
i still got an f1 of .89.  So I will just go with the second model.'''

"""I want to see what happens if ai narrow my dictionary around my best params
"""
#using the dictionary
param_dict = {"n_estimators":[200, 300, 400, 500, 600],
              "max_depth":[8, 10, 12, 14]}

#grid searching 
gs2 = GridSearchCV(rf, param_dict, scoring = "f1_macro", 
                  n_jobs = -1, verbose = 2)
#fitting on the search 
gs2.fit(data, y)
#fitting on the model of choice 
rf.fit(data, y)


Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   21.9s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   38.6s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

The spreadsheet 'newapplicants.csv' contains information on two new mortgage applicants, the first black, the second non-black, but who otherwise share identical characteristics, namely, they both have monthly debt-to-income and housing expenses to income ratio of .25, both have a loan to property value ratio of .951, both have a category 4 credit and mortgage score, neither has a public record of credit problems, neither was denied mortgage insurance, both are self-employed, both are single, neither graduated from high school, both have occupations with an industry unemployment rate of 9, and neither is purchasing a condo.

Read in the new applicant information and apply the chosen model to predict whether each applicant will be denied or not, and print out the predictions.

In [19]:
test = pd.read_csv("/content/drive/My Drive/newapplicants.csv")

test.head()

,p_irat,black,hse_inc,loan_val,ccred,mcred,pubrec,denpmi,selfemp,single,hischl,probunmp,condo,ltv_med,ltv_high
0,0.25,1,0.25,0.951,4,4,0,0,1,1,0,9,0,0,1
1,0.25,0,0.25,0.951,4,4,0,0,1,1,0,9,0,0,1


In [20]:
test_pred =  rf.predict(test)
print(test_pred)

[1 1]


In [21]:
print(sum(data.black))
print(data.shape)
print(339/2380)
print(sum(y))
print(285/2380)

339
(2380, 15)
0.14243697478991596
285
0.11974789915966387


Do the predictions differ between the black and non-black applicant, and what if anything does this suggest about the fairness of the mortgage application process?

type your answer in this cell
NO